In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 176ms/step - loss: 0.7390 - accuracy: 0.5728
Epoch 2/30
58/58 [==============================] - 10s 175ms/step - loss: 0.6150 - accuracy: 0.6550
Epoch 3/30
58/58 [==============================] - 10s 174ms/step - loss: 0.5655 - accuracy: 0.7108
Epoch 4/30
58/58 [==============================] - 10s 176ms/step - loss: 0.4830 - accuracy: 0.7601
Epoch 5/30
58/58 [==============================] - 10s 176ms/step - loss: 0.4162 - accuracy: 0.8127
Epoch 6/30
58/58 [==============================] - 10s 176ms/step - loss: 0.3602 - accuracy: 0.8456
Epoch 7/30
58/58 [==============================] - 10s 174ms/step - loss: 0.3193 - accuracy: 0.8576
Epoch 8/30
58/58 [==============================] - 10s 172ms/step - loss: 0.2321 - accuracy: 0.9003
Epoch 9/30
58/58 [==============================] - 10s 176ms/step - loss: 0.1966 - accuracy: 0.9179
Epoch 10/30
58/58 [==============================] - 10s 175ms/step - loss: 0.1449 - accura

58/58 [==============================] - 10s 178ms/step - loss: 0.0849 - accuracy: 0.9595
Epoch 22/30
58/58 [==============================] - 10s 180ms/step - loss: 0.0618 - accuracy: 0.9781
Epoch 23/30
58/58 [==============================] - 10s 177ms/step - loss: 0.0512 - accuracy: 0.9825
Epoch 24/30
58/58 [==============================] - 10s 177ms/step - loss: 0.0546 - accuracy: 0.9825
Epoch 25/30
58/58 [==============================] - 10s 177ms/step - loss: 0.0328 - accuracy: 0.9945
Epoch 26/30
58/58 [==============================] - 10s 176ms/step - loss: 0.0345 - accuracy: 0.9890
Epoch 27/30
58/58 [==============================] - 10s 179ms/step - loss: 0.0506 - accuracy: 0.9781
Epoch 28/30
58/58 [==============================] - 10s 177ms/step - loss: 0.0541 - accuracy: 0.9803
Epoch 29/30
58/58 [==============================] - 10s 176ms/step - loss: 0.0595 - accuracy: 0.9759
Epoch 30/30
58/58 [==============================] - 10s 177ms/step - loss: 0.0483 - accuracy:

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 18s 313ms/step - loss: 0.6432 - accuracy: 0.6068
Epoch 2/30
58/58 [==============================] - 18s 312ms/step - loss: 0.4300 - accuracy: 0.8018
Epoch 3/30
58/58 [==============================] - 18s 313ms/step - loss: 0.3535 - accuracy: 0.8510
Epoch 4/30
58/58 [==============================] - 18s 313ms/step - loss: 0.2284 - accuracy: 0.9200
Epoch 5/30
58/58 [==============================] - 18s 314ms/step - loss: 0.2100 - accuracy: 0.9321
Epoch 6/30
58/58 [==============================] - 18s 316ms/step - loss: 0.1204 - accuracy: 0.9639
Epoch 7/30
58/58 [==============================] - 18s 311ms/step - loss: 0.1038 - accuracy: 0.9660
Epoch 8/30
58/58 [==============================] - 18s 315ms/step - loss: 0.0547 - accuracy: 0.9836
Epoch 9/30
58/58 [==============================] - 18s 313ms/step - loss: 0.0825 - accuracy: 0.9759
Epoch 10/30
58/58 [==============================] - 18s 312ms/step - loss: 0.0356 - accura

58/58 [==============================] - 18s 309ms/step - loss: 0.0045 - accuracy: 0.9978
Epoch 22/30
58/58 [==============================] - 18s 307ms/step - loss: 0.0031 - accuracy: 0.9989
Epoch 23/30
58/58 [==============================] - 17s 294ms/step - loss: 0.0029 - accuracy: 0.9989
Epoch 24/30
58/58 [==============================] - 17s 300ms/step - loss: 0.0049 - accuracy: 0.9989
Epoch 25/30
58/58 [==============================] - 18s 309ms/step - loss: 0.0016 - accuracy: 0.9989
Epoch 26/30
58/58 [==============================] - 18s 307ms/step - loss: 0.0024 - accuracy: 0.9978
Epoch 27/30
58/58 [==============================] - 18s 308ms/step - loss: 0.0032 - accuracy: 0.9978
Epoch 28/30
58/58 [==============================] - 18s 308ms/step - loss: 0.0023 - accuracy: 0.9989
Epoch 29/30
58/58 [==============================] - 18s 308ms/step - loss: 0.0025 - accuracy: 0.9978
Epoch 30/30
58/58 [==============================] - 18s 312ms/step - loss: 0.0029 - accuracy: